In [1]:
'''
    Import packages and modules from Python Standard Library and Third party libraries.
'''
#Import from python standard library
import os

#Import from third party libraries 
import cv2
import glob
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from skimage.color import gray2rgb, rgb2gray
from tensorflow import keras
import timeit
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

In [2]:
def read_boxes():
    
    '''
        required directory hierarchy: current_directory/output/file.txt
    '''
    #Read files and get a dictionary with image 'name.png' as key and 'bounding box coordinates' as value
    #Each file contains name.png and bounding box coordinates in each row
    allBoxes = {}
    files = ['FallingDown_01.txt','FallingDown_02.txt','Standing_01.txt','Standing_02.txt']
    path = './output/'

    for filename in files:
        boxes = {}
        with open(path+filename,'r') as file:
            for line in file:
                tokens = line.split(' ',1)
                name,box = tokens[0].strip(), tokens[1].strip()
                boxes[name] = box
        key = filename.split('.')[0]
        allBoxes[key]=boxes
    
    return allBoxes

In [3]:
def prepare_data():
    '''
     required directory hierarchy: current_directory/output/DIRs
    '''
    
    allBoxes = read_boxes() 
    #define list to store the dataset 
    dataset = []
    #define list to store the data labels 
    dataLabels = []
    #list of main-directory names
    DIRs = ['FallingDown_01','FallingDown_02','Standing_01','Standing_02']
    #iterate over each main-directory name
    for DIR in DIRs:
        #join path with each main-directory name
        path = './output/'+str(DIR)
        print('DIR',DIR)
        #Extract the path of all sub-directories inside each main-directory 
        for directory in glob.glob(path):
            sub_path = os.path.join(directory+'/*.png')
            #extract path of all images in sub-directory
            image_paths = glob.glob(sub_path)
            #split the directory path and extract directory name
            labels = directory.split('\\')[-1]
            #Iterate over each image path to read the image 
            for image_path in image_paths:
                name = os.path.basename(image_path)
                try:
                    box = allBoxes[DIR][name]
                    box_tokens = [ tk for tk in box.strip('[,]').split(' ') if tk is not '']
                    x1 = box_tokens[0].strip(' ')
                    y1 = box_tokens[1].strip(' ')
                    x2 = box_tokens[2].strip(' ')
                    y2 = box_tokens[3].strip(' ')
                    box = x1,y1,x2,y2
                    x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
                    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
                    image = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)
                    #print(image.shape)
                    image = image[y1:y2,x1:x2]
                    image = cv2.resize(image,dsize=(300,300), interpolation= cv2.INTER_AREA)
                    output_path = path+'/cropped/'+name+'.png'
                    cv2.imwrite(output_path,cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
                    #print(image.shape)
                    #convert to rgb if grayscale image 
                    if len(image.shape)==2:
                        image = gray2rbg(image)
                    if DIR is 'FallingDown_01' or DIR is 'FallingDown_02':
                        #append image to dataset list  
                        dataset.append(image)
                        dataLabels.append(int(1))
                    elif DIR=='Standing_01' or DIR=='Standing_02':
                        dataset.append(image)
                        dataLabels.append(int(0))
                except KeyError:
                    print('Key Error')

    #convert dataset and data_labels to numpy array        
    dataset = np.array(dataset)
    dataLabels = np.array(dataLabels)
    
    return dataset, dataLabels

In [4]:
#methods to rescale images
def rescaleImage(image, label):
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    return image, label

In [5]:
def getTestimage(dataset, dataLabels):
    '''
        insert batch =1 in "test_dataset.map(rescaleImage).shuffle(buffer_size=1024).batch(4)"  to get one image
    '''
    
    #shuffle dataset and split dataset into train, test and validation sets
    dataset, dataLabels = shuffle(dataset, dataLabels, random_state=1236)

    #train_x, train_y = dataset[:1000], dataLabels[:1000]
    test_x, test_y   = dataset[1000:1400], dataLabels[1000:1400]
    #valid_x, valid_y = dataset[1400:], dataLabels[1400:]
    
    #convert train, test and validation sets to tensorflow datasets 
    test_dataset   = tf.data.Dataset.from_tensor_slices((test_x, test_y))
    
    #map test dataset to preprocessing function, shuffle, 
    test_dataset  = test_dataset.map(rescaleImage).shuffle(buffer_size=1024).batch(30)
    image, label  = test_dataset.as_numpy_iterator().next()
    
    return image, label

In [6]:
def action_detection_segmentation(image, model):
    
    
    #inference on single image
    prediction = model.predict(image)
    
    # Apply a sigmoid since our model returns logits
    prediction = tf.nn.sigmoid(prediction)
    prediction = tf.where(prediction < 0.6, 0, 1)
    prediction =  prediction.numpy().ravel()
    
    return prediction

In [7]:
dataset, dataLabels = prepare_data()
print('Dataset Shape: {0} Labels Shape: {1}'.format(dataset.shape, dataLabels.shape))

DIR FallingDown_01
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
Key Error
DIR FallingDown_02
Key Error
Key Error
Key Error
DIR Standing_01
DIR Standing_02
Key Error
Key Error
Key Error
Key Error
Key Error
Dataset Shape: (1625, 300, 300, 3) Labels Shape: (1625,)


In [9]:
image, label = rescaleImage(dataset[0], dataLabels[0])
image2 = tf.expand_dims(image, axis=0)

In [10]:
import time
start_time = time.time()
model_path="./fall_model/model_6.h5"
#load model
ResNetmodel = keras.models.load_model(model_path)
elapsed_time = time.time() - start_time
print("Iteration time: %0.4fs" % elapsed_time)

Iteration time: 3.0099s


In [11]:
import time
start_time = time.time()
model_path="./fall_model/model_5.h5"
#load model
EffNetmodel = keras.models.load_model(model_path)
elapsed_time = time.time() - start_time
print("Iteration time: %0.4fs" % elapsed_time)

Iteration time: 4.6981s


In [12]:

warnings.filterwarnings("ignore")
start_time = time.time()
# image , label = getTestimage(dataset, dataLabels)
prediction = action_detection_segmentation(image2,ResNetmodel)
elapsed_time = time.time() - start_time
print("Iteration time: %0.4fs" % elapsed_time)

Iteration time: 3.4772s


In [ ]:
warnings.filterwarnings("ignore")
start_time = time.time()
# image , label = getTestimage(dataset, dataLabels)
prediction = action_detection_segmentation(image2,model5)
elapsed_time = time.time() - start_time
print("Iteration time: %0.4fs" % elapsed_time)

In [ ]:
times = []
for i in range(30):
    start_time = time.time()
    one_prediction = action_detection_segmentation(image2, model2)
    delta = (time.time() - start_time)
    times.append(delta)
mean_delta = np.array(times).mean()
fps = 1 / mean_delta
print('average(sec):{:.2f},fps:{:.2f}'.format(mean_delta, fps))

In [ ]:
times = []
for i in range(30):
    start_time = time.time()
    one_prediction = action_detection_segmentation(image2, model5)
    delta = (time.time() - start_time)
    times.append(delta)
mean_delta = np.array(times).mean()
fps = 1 / mean_delta
print('average(sec):{:.2f},fps:{:.2f}'.format(mean_delta, fps))

In [14]:
execution = timeit.timeit(lambda: action_detection_segmentation(image2, ResNetmodel), number=30)
FPS = 30/execution 
print('Execution time for 30 runs: {:.2f}, FPS: {:.2f}'.format(execution, FPS))

Execution time for 30 runs: 1.23, FPS: 24.29


In [15]:
execution = timeit.timeit(lambda: action_detection_segmentation(image2, EffNetmodel), number=30)
FPS = 30/execution 
print('Execution time for 30 runs: {:.2f}, FPS: {:.2f}'.format(execution, FPS))

Execution time for 30 runs: 1.16, FPS: 25.78
